In [53]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm

In [54]:
raw_survey_data = pd.read_csv('./raw data.csv', low_memory = False)
prolific_data = pd.read_csv('./prolific data.csv')

Step1: filter unfinished, time-out samples and fail to pass example participants by prolific summary

Note*: there are two participants didn't fill participant id correctly, after matching prolific data file and raw data file, we delete these two subjects.

In [55]:
approve_prolific = prolific_data[prolific_data.Status == 'APPROVED']
print(approve_prolific.shape)
approve_prolific.to_csv('check_prolific.csv')
survey_data = raw_survey_data
survey_data['Q1324'] = survey_data['Q1324'].str.lower()
survey_data['Q1324'] = survey_data.Q1324.str.split('@').str[0]
df = survey_data[survey_data.set_index(['Q1324']).index.isin(approve_prolific.set_index(['Participant id']).index)]
colum_name = survey_data[:1]
df1 = pd.concat([colum_name, df], ignore_index = True)
print(df1.shape)
df1.to_csv('check.csv')

(744, 20)
(745, 3522)


Step2: tansfer the demogaphic data
Note*: 1 for frequently buy online; 1 for not familar with shampoo; 1 for not clear image;1 for fully display; 1 for not slow internet; 1 for youunger than 20 years old; 1 for male(female, non-binary, perfer self-discrible, not to say)

In [56]:
df2 = df1.replace({'demo_buy_online': {'Yes': 1, 'No': 0},'demo_familarity_1': {'Not familiar at all': 1, 'Slightly familiar': 2,'Moderately familiar': 3,'Very familiar': 4,'Extremely familiar': 5},
                 'demo_img_clear_1': {'Not clear at all': 1, 'Slightly clear': 2,'Moderately clear': 3,'Very clear': 4,'Extremely clear': 5},'demo_img_complete': {'Yes': 1, 'Partial': 0, 'No': 0},
                 'demo_internet_1': {'Not slow at all': 1, 'Slightly slow': 2,'Moderately slow': 3,'Very slow': 4,'Extremely slow': 5},
                 'Qage': {'Younger than 20 years': 1, 'Between 20 and 29 years': 2,'Between 30 and 39 years': 3,'Between 40 and 49 years': 4,'Between 50 and 59 years': 5, '60 years or older': 6},
                 'Qgender': {'Male': 0, 'Female': 1,'Non-binary': 0,'Prefer to self-describe': 0,'Prefer not to say': 0}
                })
print(df2.shape)

(745, 3522)


Step3: remove subjects who state the internet is exteremely slow


In [57]:
df3 = df2.drop(df2[df2.demo_internet_1 == 5].index)
print(df3.shape)

(741, 3522)


Step4: 1). divide subjects into two experiment condition: no time pressure(0) or time pressure(1) 
       2). indicate version, either original(0) or changed(1)
       3). indicate correctly find the target or not
       4). indicate number of selected region
       5). indicate search time, by first click time, last click time, page submission 
Note*: q_change3_shelf17 set to like/dislike --> on/off

In [58]:
# df3['condition'] = df3['Q572_Page Submit'].isnull()*1
df3 = df3.replace(['Neutral', 'Dislike'], 'Off') 
df3 = df3.replace('Like', 'On') 

In [59]:
def transfer(df, packNum, shelfNum, verNum, bottleNum):
    df_copy = copy.deepcopy(df)
    version_column = ['Q937_Page Submit', 'Q944_Page Submit','Q959_Page Submit']
    original_no_time_colum = ['Q902', 'Q816', 'Q1169']
    change_no_time_column = ['Q1048', 'Q992', 'Q1102']
    for n in tqdm(range(1, packNum + 1)):
        df_copy['version' + str(n)] = df[version_column[n-1]].isnull() * 1 
        conditions = []
        choices = [1] * shelfNum * verNum
        column_name_start = 'q_' + 'original' + str(n) + '_' + 'shelf1_1'
        column_loc_start = df.columns.get_loc(column_name_start)
        for i in range(1, shelfNum + 1):
            for m in range(verNum):
                if m==0:
                    version = 'original'
                else:
                    version = 'change'
                column_name = 'q_' + version + str(n) + '_' + 'shelf' + str(i) +' - target'
                shelf_target = df.where(df.eq(column_name)).stack().index.tolist()[-1][-1]
                column_loc = df.columns.get_loc(shelf_target)
                conditions.append(df.iloc[:, column_loc].eq('On'))
        df_copy['count'+ str(n)] = df.iloc[:, column_loc_start:int(column_loc_start + shelfNum * verNum * bottleNum + 12)].eq('On').sum(axis = 1)
        df_copy['correct'+ str(n)] = np.select(conditions, choices, default = 0)
        df_copy['time_first click' + str(n)] = df[original_no_time_colum[n - 1] + '_First Click'].combine_first(df[change_no_time_column[n - 1] + '_First Click'])
        df_copy['time_last click' + str(n)] = df[original_no_time_colum[n - 1] + '_Last Click'].combine_first(df[change_no_time_column[n - 1] + '_Last Click'])
        df_copy['time_click count' + str(n)] = df[original_no_time_colum[n - 1] + '_Click Count'].combine_first(df[change_no_time_column[n - 1] + '_Click Count'])
    return(df_copy)


In [60]:
packNum = 3
shelfNum = 20
verNum = 2
bottleNum = 27
df4 = transfer(df3, packNum, shelfNum, verNum, bottleNum).reset_index()
print(df4)
# df4['Q948_First Click'].to_csv('df4_original4.csv')
# df4['Q1048_First Click'].to_csv('df4_change1.csv')

100%|█████████████████████████████████████████████| 3/3 [01:10<00:00, 23.42s/it]

     index        StartDate          EndDate         Status        IPAddress  \
0        0       Start Date         End Date  Response Type       IP Address   
1        1  24/12/2024 2:08  24/12/2024 2:11     IP Address  185.241.224.233   
2        2  24/12/2024 2:08  24/12/2024 2:12     IP Address     83.98.59.146   
3        3  24/12/2024 2:07  24/12/2024 2:12     IP Address    88.98.240.100   
4        4  24/12/2024 2:10  24/12/2024 2:14     IP Address   81.158.156.157   
..     ...              ...              ...            ...              ...   
736    740  24/12/2024 3:30  24/12/2024 3:34     IP Address    185.58.164.46   
737    741  24/12/2024 3:30  24/12/2024 3:35     IP Address     31.51.53.188   
738    742  24/12/2024 3:30  24/12/2024 3:35     IP Address    92.26.192.201   
739    743  24/12/2024 3:30  24/12/2024 3:35     IP Address      2.24.84.207   
740    744  24/12/2024 3:31  24/12/2024 3:38     IP Address     77.100.2.174   

     Progress  Duration (in seconds)  F

Step5: transfer data into task level

In [61]:
df4.to_csv('process_raw.csv')
# df4 = pd.read_csv('process_raw.csv')

In [62]:
df4 = pd.read_csv('process_raw.csv')
flow_dict = {
    'FL_1237': 1 , 'FL_1252': 2, 'FL_1540': 3}

df4_copy_copy = copy.deepcopy(df4)

# Extract the flow codes into separate columns
df4_copy_copy[['flow1', 'flow2', 'flow3']] = df4['FL_1236_DO'].str.extract(
    r'(FL_\d+)\|(FL_\d+)\|(FL_\d+)'
)

df4_copy_copy[['task1', 'task2', 'task3']] = df4['FL_1236_DO'].str.extract(
    r'(FL_\d+)\|(FL_\d+)\|(FL_\d+)'
)

# Replace the flow strings with their corresponding numeric values using flow_dict
df4_copy_copy[['flow1', 'flow2', 'flow3']] = df4_copy_copy[[
    'flow1', 'flow2', 'flow3']].replace(flow_dict)
print(df4_copy_copy)
df4_copy_copy.to_csv('process_raw_2.csv')

/var/folders/81/5bx135bd1sn42gnywk_3s_pc0000gn/T/ipykernel_59791/2193761197.py:1: DtypeWarning: Columns (6,7,8,11,12,13,14,15,16,19,20,21,22,24,25,26,27,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,66,67,68,69,70,71,72,73,74,75,76,77,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,727,728,729,730,731,732,733,734,735,736,737,738,1279,1280,1281,1282,1283,1284,1285,1286,1287,1288,1289,1290,1831,1832,1833,1834,1835,1836,1837,1838,1839,1840,1841,1842,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,2383,2384,2385,2386,2387,2388,2389,2390,2391,2392,2393,2394,2935,2936,2937,2938,2939,2940,2941,2942,2943,2944,2945,2946,3487,3488,3489,3490,3491,3492,3493,3494,3495,3496,3497,3499,3500,3501,3502,3503,3504,350

     Unnamed: 0  index        StartDate          EndDate         Status  \
0             0      0       Start Date         End Date  Response Type   
1             1      1  24/12/2024 2:08  24/12/2024 2:11     IP Address   
2             2      2  24/12/2024 2:08  24/12/2024 2:12     IP Address   
3             3      3  24/12/2024 2:07  24/12/2024 2:12     IP Address   
4             4      4  24/12/2024 2:10  24/12/2024 2:14     IP Address   
..          ...    ...              ...              ...            ...   
736         736    740  24/12/2024 3:30  24/12/2024 3:34     IP Address   
737         737    741  24/12/2024 3:30  24/12/2024 3:35     IP Address   
738         738    742  24/12/2024 3:30  24/12/2024 3:35     IP Address   
739         739    743  24/12/2024 3:30  24/12/2024 3:35     IP Address   
740         740    744  24/12/2024 3:31  24/12/2024 3:38     IP Address   

           IPAddress  Progress  Duration (in seconds)  Finished  \
0         IP Address  Progress  

In [63]:
def extract_shelf(row):
    if row:
        parts = row.split('_shelf')
        if len(parts) > 1:
#             print(parts[1].split('|')[0])
            return parts[1].split('|')[0]
    return None

for i in range(1, 4):
    original_col = f'SearchTask{i}:originalbottle{i}Question_DO'
    change_col = f'SearchTask{i}:changebottle{i}Question_DO'
    original_shelf_col = f'task{i}_original_shelf'
    change_shelf_col = f'task{i}_change_shelf'
#     df4_copy_copy[original_shelf_col] = df4_copy_copy[original_col].apply(lambda x: extract_shelf(x[0]))
#     df4_copy_copy[change_shelf_col] = df4_copy_copy[change_col].apply(lambda x: extract_shelf(x[1]))
    df4_copy_copy[original_shelf_col] = df4_copy_copy[original_col].apply(lambda x: extract_shelf(str(x)))
    df4_copy_copy[change_shelf_col] = df4_copy_copy[change_col].apply(lambda x: extract_shelf(str(x)))
# Display the modified DataFrame with new columns
print(df4_copy_copy)

     Unnamed: 0  index        StartDate          EndDate         Status  \
0             0      0       Start Date         End Date  Response Type   
1             1      1  24/12/2024 2:08  24/12/2024 2:11     IP Address   
2             2      2  24/12/2024 2:08  24/12/2024 2:12     IP Address   
3             3      3  24/12/2024 2:07  24/12/2024 2:12     IP Address   
4             4      4  24/12/2024 2:10  24/12/2024 2:14     IP Address   
..          ...    ...              ...              ...            ...   
736         736    740  24/12/2024 3:30  24/12/2024 3:34     IP Address   
737         737    741  24/12/2024 3:30  24/12/2024 3:35     IP Address   
738         738    742  24/12/2024 3:30  24/12/2024 3:35     IP Address   
739         739    743  24/12/2024 3:30  24/12/2024 3:35     IP Address   
740         740    744  24/12/2024 3:31  24/12/2024 3:38     IP Address   

           IPAddress  Progress  Duration (in seconds)  Finished  \
0         IP Address  Progress  

In [77]:
# Initialize the final dataframe
df5 = pd.DataFrame(columns = [
    'ID', 'ProlificID', 'DemoBuy', 'DemoFamilarity', 'DemoClear', 
    'DemoComplete', 'DemoInternet', 'DemoAge', 'DemoGender', 
    'TaskID', 'Version', 'ClickRegionCount', 'CorrectTarget', 
    'TimeFirstClick', 'TimeLastClick', 'TimeClickCount', 'Flow'
])

# Iterate over the rows of df4_copy_copy
for i in tqdm(range(1,df4_copy_copy.shape[0])):
    # Extract demographic and other static information
    df_sub_level = df4_copy_copy.loc[i, ['Q1324', 'demo_buy_online', 'demo_familarity_1', 'demo_img_clear_1', 'demo_img_complete', 'demo_internet_1', 'Qage', 'Qgender']]
    df_sub_level = df_sub_level.to_frame().T
    df_sub_task_level = pd.DataFrame(np.repeat(df_sub_level.values, packNum, axis=0))
    df_id = pd.DataFrame(np.repeat(i, packNum))
    df_sub_task_level = pd.concat([df_id, df_sub_task_level], axis=1, ignore_index=True)
    df_sub_task_level.columns = ['ID', 'ProlificID', 'DemoBuy', 'DemoFamilarity', 'DemoClear', 'DemoComplete', 'DemoInternet', 'DemoAge', 'DemoGender']
    
    # Initialize task level dataframe
    df_task_level_new = pd.DataFrame(columns=['Version', 'ClickRegionCount', 'CorrectTarget', 'TimeFirstClick', 'TimeLastClick', 'TimeClickCount', 'Flow'])
    
    # Extract the flow values for this row (flow1 to flow8)
    flow_values = df4_copy_copy.loc[i, ['flow1', 'flow2', 'flow3']].values
    shelf_values = df4_copy_copy.loc[i, ['task1_original_shelf', 'task1_change_shelf', 'task2_original_shelf', 'task2_change_shelf','task3_original_shelf', 'task3_change_shelf']].values
    # Iterate over each task (package) and extract task-level information
    df_task_id = pd.DataFrame(np.arange(1, packNum + 1, dtype=int))
    for j in range(1, packNum + 1):
        # Extract task-level data
        df_task_level = df4_copy_copy.loc[i, [
            'version' + str(j),
            'count' + str(j),
            'correct' + str(j),
            'time_first click' + str(j),
            'time_last click' + str(j),
            'time_click count' + str(j),
            'task' + str(j) + '_original_shelf',
            'task' + str(j) + '_change_shelf'
        ]].to_frame().T

        df_task_level.columns = ['Version', 'ClickRegionCount', 'CorrectTarget', 'TimeFirstClick', 'TimeLastClick', 'TimeClickCount', 'OriginalShelf', 'ChangeShelf']
        
        # Find the flow column corresponding to the current task number j
        flow_position = np.where(flow_values == j)[0][0] + 1  # +1 to get flow column index (flow1 = 1, flow2 = 2, ..., flow8 = 8)
        # Assign the flow column index (flow_position) as the flow value
        df_task_level['Flow'] = flow_position
        # Determine which shelf value to keep
        df_task_level['Shelf'] = df_task_level.apply(lambda row: row['OriginalShelf'] if pd.notnull(row['OriginalShelf']) else row['ChangeShelf'], axis=1)
            # Append the task-level data to the new dataframe
        df_task_level_new = pd.concat([df_task_level_new, df_task_level], axis=0, ignore_index=True)

    # Combine task IDs and task-level details
    df_task_level_concat = pd.concat([df_task_id, df_task_level_new], axis=1, ignore_index=True)
    df_task_level_concat.columns = ['TaskID', 'Version', 'ClickRegionCount', 'CorrectTarget', 'TimeFirstClick', 'TimeLastClick', 'TimeClickCount', 'Flow', 'OriginalShelf', 'ChangeShelf', 'Shelf']

    # Combine subject-level and task-level data
    df5_part = pd.concat([df_sub_task_level, df_task_level_concat], axis=1, ignore_index=True)

    df5_part.columns = ['ID', 'ProlificID', 'DemoBuy', 'DemoFamilarity', 'DemoClear', 'DemoComplete', 'DemoInternet', 'DemoAge', 'DemoGender', 'TaskID', 'Version', 'ClickRegionCount', 'CorrectTarget', 'TimeFirstClick', 'TimeLastClick', 'TimeClickCount', 'Flow','OriginalShelf', 'ChangeShelf', 'Shelf']
    # Append to the final dataframe
    df5 = pd.concat([df5, df5_part], axis=0, ignore_index=True)

# Save the final dataframe to CSV
print(df5)
df5.to_csv('dataclean.csv', index=False)

100%|█████████████████████████████████████████| 740/740 [00:13<00:00, 53.88it/s]

       ID                ProlificID DemoBuy DemoFamilarity DemoClear  \
0       1  5c094b4b1907db00016017d7       1              4         5   
1       1  5c094b4b1907db00016017d7       1              4         5   
2       1  5c094b4b1907db00016017d7       1              4         5   
3       2  60148ce8ea15974d855c06d5       1              5         4   
4       2  60148ce8ea15974d855c06d5       1              5         4   
...   ...                       ...     ...            ...       ...   
2215  739  6730f2e20f5915ff4ad2c07d       1              4       5.0   
2216  739  6730f2e20f5915ff4ad2c07d       1              4       5.0   
2217  740  5caf6864971a7b0012dc7562       1              5       4.0   
2218  740  5caf6864971a7b0012dc7562       1              5       4.0   
2219  740  5caf6864971a7b0012dc7562       1              5       4.0   

     DemoComplete DemoInternet DemoAge DemoGender TaskID Version  \
0               1            1       2          1      1       0   